In [1]:
import os
import sys
import math
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt
import catboost as cb
import pickle
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, precision_score, recall_score, accuracy_score, f1_score
from sklearn.metrics import make_scorer, precision_recall_curve, auc
#from sklearn.preprocessing import StandardScaler, MinMaxScaler
#from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from bayes_opt import BayesianOptimization
#from sklearn.experimental import enable_halving_search_cv 
#from sklearn.model_selection import HalvingRandomSearchCV
#from sklearn.model_selection import HalvingGridSearchCV 
from collections import Counter
from imblearn.over_sampling import SMOTE
from catboost import Pool, CatBoost, CatBoostClassifier
import imblearn
print(imblearn.__version__)

0.9.1


# Menu

<a name="navegacao"></a>

## 1) [Preparação dos dados](#parte1)
- 1.1 [Leitura base principal](#principal)
- 1.2 [Leitura base mes](#mes)
- 1.3 [Leitura base hora](#hora)
- 1.4 [Merge principal e base mensal](#merge1)
- 1.5 [Merge principal e base hora](#merge2)
- 1.6 [Confere marcação](#marcacao)


## 2 [Salvando as bases de treino](#parte2)
- 2.1 [Salvando base com histórico](#comhist)
- 2.2 [Salvando base sem histórico](#semhist)


<a name="principal"></a>

## 1.1) Leitura base principal


## Leitura dos dados

In [2]:
%%time
df = pd.read_pickle("perfilpj.pkl")

Wall time: 156 ms


In [3]:
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df.dropna(inplace=True)

In [4]:
df.shape

(44691, 202)

In [5]:
df.columns

Index(['NR_CPF_CNPJ', 'DT_MVM_DWM', 'NR_CPR', 'CNAE', 'CD_NVL_RCO', 'VR_CSC',
       'VR_ATL_IMV', 'CAPITAL_SOCIAL', 'NATJUR', 'PORTE',
       ...
       'spl_idade_5_0', 'spl_idade_5_1', 'spl_idade_5_2', 'spl_idade_5_3',
       'spl_idade_5_4', 'spl_idade_5_5', 'spl_idade_5_6', 'spl_idade_5_7',
       'spl_idade_5_8', 'spl_idade_5_9'],
      dtype='object', length=202)

In [6]:
%%time
df.filter(regex='spl_idade', axis=1).describe()

Wall time: 290 ms


,spl_idade_2_0,spl_idade_2_1,spl_idade_2_2,spl_idade_2_3,spl_idade_2_4,spl_idade_2_5,spl_idade_2_6,spl_idade_2_7,spl_idade_2_8,spl_idade_2_9,...,spl_idade_5_0,spl_idade_5_1,spl_idade_5_2,spl_idade_5_3,spl_idade_5_4,spl_idade_5_5,spl_idade_5_6,spl_idade_5_7,spl_idade_5_8,spl_idade_5_9
count,44691.000000,44691.000000,44691.000000,44691.000000,44691.000000,44691.000000,44691.000000,44691.000000,44691.0,44691.000000,...,44691.000000,44691.000000,44691.000000,44691.000000,44691.000000,44691.000000,44691.000000,44691.000000,44691.0,44691.000000
mean,1.367465,2.065321,1.976514,2.626068,2.376308,2.263033,3.089134,5.673474,0.0,3.603747,...,96.302340,170.609776,183.911609,273.824571,274.890267,284.840376,430.863617,947.813517,0.0,765.101069
std,4.625612,5.907901,6.056045,7.160614,7.125054,7.191227,8.574865,11.824995,0.0,10.727324,...,332.277566,488.907964,564.184026,747.772384,824.995540,905.707844,1198.103053,1987.008023,0.0,2320.206077
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
max,18.049712,19.737150,21.207592,22.920077,24.419830,25.757389,28.072167,32.533135,0.0,48.799192,...,1384.126173,1730.657443,2071.230448,2515.012742,2946.840183,3367.089055,4175.320750,6036.910986,0.0,16635.335692


In [7]:
%%time
#df.filter(regex='spl_', axis=1).columns[60:]
df.dtypes.head(15)

Wall time: 0 ns


NR_CPF_CNPJ               object
DT_MVM_DWM        datetime64[ns]
NR_CPR                   float64
CNAE                      object
CD_NVL_RCO                object
VR_CSC                   float64
VR_ATL_IMV               float64
CAPITAL_SOCIAL           float64
NATJUR                    object
PORTE                    float64
QTDSOCIOS                float64
FRAUDES                  float64
IDADE                    float64
TEMPO_CONTA              float64
RENDA                    float64
dtype: object

In [10]:
pd.crosstab(df.DT_MVM_DWM, df.FRAUDES)

FRAUDES,0.0,1.0
DT_MVM_DWM,,
2022-01-01,17,14
2022-02-01,9383,1118
2022-03-01,146,28
2022-04-01,115,22
2022-05-01,11075,798
2022-06-01,951,40
2022-07-01,717,24
2022-08-01,11020,181
2022-09-01,19,10


In [11]:
%%time
df[['FRAUDES','NR_CPF_CNPJ']].groupby('FRAUDES').count()

Wall time: 15 ms


,NR_CPF_CNPJ
FRAUDES,
0.0,42453
1.0,2238


In [12]:
%%time
df[['CNAE','NR_CPF_CNPJ']].groupby('CNAE').count()

Wall time: 18.9 ms


,NR_CPF_CNPJ
CNAE,
b'9',9
b'A',331
b'B',100
b'C',5618
b'D',9
b'E',122
b'F',1737
b'G',21920
b'H',2475


In [44]:
df['NR_CPR']      = df['NR_CPR'].astype(int)
df['PORTE']       = df['PORTE'].astype(int)
df['NATJUR']      = df['NATJUR'].astype(int) 
df['CNAE']        = df.CNAE.astype("category").cat.codes
df['CD_NVL_RCO']  = df.CD_NVL_RCO.astype("category").cat.codes
df['QTDSOCIOS']   = df['QTDSOCIOS'].astype(int) 


df.loc[ (df['IDADE']<1)          ,'IDADE']          = 1
df.loc[ (df['TEMPO_CONTA']<1)    ,'TEMPO_CONTA']    = 1
df.loc[ (df['RENDA']<1)          ,'RENDA']          = 1
df.loc[ (df['SCORE_SERASA']<1)   ,'SCORE_SERASA']   = 1
df.loc[ (df['VR_DSP']<1)         ,'VR_DSP']         = 1
df.loc[ (df['VR_TRN']<1)         ,'VR_TRN']         = 1
df.loc[ (df['VR_CSC']<1)         ,'VR_CSC']         = 1
df.loc[ (df['CAPITAL_SOCIAL']<1) ,'CAPITAL_SOCIAL'] = 1
df.loc[ (df['VR_ATL_IMV']<1)     ,'VR_ATL_IMV']     = 1


df['ln_idade']      = np.log(df['IDADE'])
df['ln_tempo']      = np.log(df['TEMPO_CONTA'])
df['ln_renda']      = np.log(df['RENDA'])
df['ln_serasa']     = np.log(df['SCORE_SERASA'])
df['ln_disponivel'] = np.log(df['VR_DSP'])
df['ln_transacao']  = np.log(df['VR_TRN'])
df['ln_capsoc_sic'] = np.log(df['VR_CSC'])
df['ln_capsoc_rec'] = np.log(df['CAPITAL_SOCIAL'])
df['ln_ativoimov']  = np.log(df['VR_ATL_IMV'])


df['diff_idade_tempo'] = df['ln_idade']-df['ln_tempo']
df['diff_renda_disp']  = df['ln_renda']-df['ln_disponivel']
df['diff_renda_tran']  = df['ln_renda']-df['ln_transacao']
df['diff_renda_cap']   = df['ln_renda']-df['ln_capsoc_sic']
df['diff_renda_ativo'] = df['ln_renda']-df['ln_ativoimov']

## Modelo sem histórico foi treinado com as variáveis na seguinte ordem:
['faixa_hora', 'vl_medio_mes_atual', 'dif_vl_1', 'tres_prim_dig_codbarras', 'pagador_pf', 'dif_vl_4', 'dia_do_mes', 'qtd_operacoes_mes_corrente', 'vl_medio_dia_corrente', 'sec_dig', 'qtd_operacoes_dia_corrente', 'qtd_trn_60min', 'centavos', 'dia_da_semana']

In [45]:
%%time
files = df.columns
selected_files = files.str.contains('ln_|diff_|CNAE|NR_CPR|CD_NVL_RCO|NATJUR|PORTE|QTDSOCIOS')
atributes = files[selected_files]

Wall time: 991 µs


In [46]:
atributes

Index(['NR_CPR', 'CNAE', 'CD_NVL_RCO', 'NATJUR', 'PORTE', 'QTDSOCIOS',
       'ln_idade', 'ln_tempo', 'ln_renda', 'ln_serasa', 'ln_disponivel',
       'ln_transacao', 'ln_capsoc_sic', 'ln_capsoc_rec', 'ln_ativoimov',
       'ln_socios', 'diff_idade_tempo', 'diff_renda_disp', 'diff_renda_tran',
       'diff_renda_cap', 'diff_renda_ativo'],
      dtype='object')

In [47]:
df[atributes].dtypes

NR_CPR                int32
CNAE                   int8
CD_NVL_RCO             int8
NATJUR                int32
PORTE                 int32
QTDSOCIOS             int32
ln_idade            float64
ln_tempo            float64
ln_renda            float64
ln_serasa           float64
ln_disponivel       float64
ln_transacao        float64
ln_capsoc_sic       float64
ln_capsoc_rec       float64
ln_ativoimov        float64
ln_socios           float64
diff_idade_tempo    float64
diff_renda_disp     float64
diff_renda_tran     float64
diff_renda_cap      float64
diff_renda_ativo    float64
dtype: object

## SMOTE: Synthetic Minority Oversampling Technique

In [48]:
y0 = df['FRAUDES']
X0 = df[atributes]

In [49]:
#import joblib
#scaler = joblib.load("scaler.saved") 

# Dado original
#X_ = scaler.transform(X)

In [50]:
#https://github.com/scikit-learn-contrib/imbalanced-learn
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
from collections import Counter
from imblearn.over_sampling import SMOTE

over = SMOTE(sampling_strategy=0.60)
under = RandomUnderSampler(sampling_strategy=0.60)
steps = [('o', over), ('u', under)]
pipeline = Pipeline(steps=steps)

In [51]:
# transform the dataset

X, y = pipeline.fit_resample(X0, y0)


In [52]:
# summarize the new class distribution
counter0 = Counter(df['FRAUDES'])
counter1 = Counter(y)

print("smote:" , counter0 ,counter1)

smote: Counter({0.0: 42453, 1.0: 2238}) Counter({0.0: 42451, 1.0: 25471})


In [53]:
print("Balanceamento SMOTE:", (21226/(25471 + 25471)) *100 )

Balanceamento SMOTE: 41.66699383612736


In [54]:
# base sintética: treino e teste
x_train  , x_test0 , y_train, y_test0 = train_test_split(X, y, test_size = 0.5, random_state=123)

# base sintética: teste e out of sample
x_test , x_out , y_test, y_out = train_test_split(x_test0, y_test0, test_size = 0.5, random_state=123)

In [55]:
#x_train.filter(regex='ln_', axis=1)


In [56]:
x_train.columns


Index(['NR_CPR', 'CNAE', 'CD_NVL_RCO', 'NATJUR', 'PORTE', 'QTDSOCIOS',
       'ln_idade', 'ln_tempo', 'ln_renda', 'ln_serasa', 'ln_disponivel',
       'ln_transacao', 'ln_capsoc_sic', 'ln_capsoc_rec', 'ln_ativoimov',
       'ln_socios', 'diff_idade_tempo', 'diff_renda_disp', 'diff_renda_tran',
       'diff_renda_cap', 'diff_renda_ativo'],
      dtype='object')

In [57]:
len(y_train)

33961

In [58]:
train_data = Pool( data= x_train, 
                   label= y_train, 
                   cat_features= [0,1,2,3,4,5])

test_data = Pool( data= x_test, 
                  label= y_test, 
                  cat_features= [0,1,2,3,4,5])

outsam_data = Pool( data=x_out, 
                    label=y_out, 
                    cat_features= [0,1,2,3,4,5])

In [59]:
from catboost import Pool, CatBoost, CatBoostClassifier


#cb = CatBoost({'iterations': 10, 'learning_rate': 0.001})
cb = CatBoostClassifier(iterations=10, learning_rate= 0.001)
cb.fit(train_data)

0:	learn: 0.6921496	total: 238ms	remaining: 2.14s
1:	learn: 0.6911090	total: 282ms	remaining: 1.13s
2:	learn: 0.6901477	total: 335ms	remaining: 781ms
3:	learn: 0.6891319	total: 386ms	remaining: 579ms
4:	learn: 0.6880991	total: 446ms	remaining: 446ms
5:	learn: 0.6870792	total: 516ms	remaining: 344ms
6:	learn: 0.6860778	total: 580ms	remaining: 249ms
7:	learn: 0.6850521	total: 698ms	remaining: 175ms
8:	learn: 0.6840362	total: 763ms	remaining: 84.8ms
9:	learn: 0.6829664	total: 829ms	remaining: 0us


In [60]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score

# get predicted labels
#pred_labels = cb.predict(test_data)

pred_labels = cb.predict(test_data,prediction_type='Class')

print("Accuracy: ", accuracy_score(y_test, pred_labels))
print("AUC-ROC: ", roc_auc_score(y_test, pred_labels))

Accuracy:  0.8836866902237926
AUC-ROC:  0.8505438808499444


In [61]:
%%time
from bayes_opt import BayesianOptimization
from sklearn.svm import SVC

lem_range = ['Newton','Gradient']

def gbm_cat( learning_rate, depth, l2_leaf_reg ,model_size_reg, rsm, leaf_estimation_method,
             iterations, bagging_temperature, eval_metric):
    model = CatBoostClassifier(
                  learning_rate           = learning_rate,
                  depth                   = int(depth),
                  l2_leaf_reg             = l2_leaf_reg,
                  model_size_reg          = model_size_reg,
                  rsm                     = rsm,
                  leaf_estimation_method  = lem_range[int(leaf_estimation_method)],
                  iterations              = int(iterations) ,  
                  bagging_temperature     = bagging_temperature,
                  eval_metric             = 'Logloss' ,  random_state=666)
    
    #model.fit(train_data, y_train, eval_set=test_data, verbose=False)
    model.fit(train_data, eval_set=test_data, verbose=False)
    pred_labels = cb.predict(test_data,prediction_type='Class')
    return roc_auc_score(y_test, pred_labels)

# scale_pos_weight
# semi-supervisionado: isolation forest, local outlier, svm

params_cat = {
    'learning_rate'           : (0.0001 ,0.666),
    'depth'                   : (1.0000 ,12.00),
    'l2_leaf_reg'             : (0.0001 ,12.00),
    'model_size_reg'          : (0.0001 ,100.0),
    'rsm'                     : (0.0001 ,0.999),
    'leaf_estimation_method'  : (0.0001 ,1.000),
    'iterations'              : (1.0000 ,200.0),
    'bagging_temperature'     : (0.0001 ,100.0),
    'eval_metric'             : (0.0001 ,1.000)
}

svm0 = BayesianOptimization(f=gbm_cat, pbounds=params_cat, random_state=123) 
svm0.maximize(init_points=30, n_iter=320, acq='ucb')

|  400      |  0.8505   |  11.61    |  7.012    |  0.345    |  139.5    |  10.58    |  0.1897   |  0.2444   |  17.83    |  0.7289   |
Wall time: 54min 44s


In [62]:
params_svm0 = svm0.max['params']
params_svm0

{'bagging_temperature': 69.6469489128676,
 'depth': 4.147532684454174,
 'eval_metric': 0.22692876841884668,
 'iterations': 110.71163904749535,
 'l2_leaf_reg': 8.633655690529778,
 'leaf_estimation_method': 0.4231641494784485,
 'learning_rate': 0.6531908797043154,
 'model_size_reg': 68.48300537551248,
 'rsm': 0.48050287639272815}

In [63]:
depth                   = int(svm0.max['params']['depth'])
iterations              = int(svm0.max['params']['iterations'])
l2_leaf_reg             = svm0.max['params']['l2_leaf_reg']
leaf_estimation_method  = lem_range[int(svm0.max['params']['leaf_estimation_method'])]
learning_rate           = svm0.max['params']['learning_rate']
model_size_reg          = int(svm0.max['params']['model_size_reg'])
rsm                     = svm0.max['params']['rsm']
bagging_temperature     = svm0.max['params']['bagging_temperature']
    

print('\n depth:'                  ,depth,
      '\n iterations:'             ,iterations,
      '\n l2_leaf_reg:'            ,l2_leaf_reg,
      '\n leaf_estimation_method:' ,leaf_estimation_method,
      '\n learning_rate:'          ,learning_rate,
      '\n model_size_reg:'         ,model_size_reg,
      '\n rsm:'                    ,rsm,
      '\n bagging_temperature:'    ,bagging_temperature)


 depth: 4 
 iterations: 110 
 l2_leaf_reg: 8.633655690529778 
 leaf_estimation_method: Newton 
 learning_rate: 0.6531908797043154 
 model_size_reg: 68 
 rsm: 0.48050287639272815 
 bagging_temperature: 69.6469489128676


In [ ]:
%%time
from catboost import Pool, CatBoost, CatBoostClassifier

cbbb= CatBoostClassifier( depth=depth, iterations=iterations, l2_leaf_reg=l2_leaf_reg, 
                           leaf_estimation_method=leaf_estimation_method, learning_rate=learning_rate, 
                           model_size_reg=model_size_reg, rsm=rsm, bagging_temperature =bagging_temperature ,  verbose=False)

cbbb.fit(x_train, y_train)

In [65]:
%%time
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score
print("Accuracy:" ,accuracy_score(  y_train, cbbb.predict(x_train))) #Accuracy: 0.998776450266302
print("F1 score:" ,f1_score(        y_train, cbbb.predict(x_train))) #F1 score: 0.37065637065637064
print("Recall:"   ,recall_score(    y_train, cbbb.predict(x_train))) #Recall: 0.25
print("Precision:",precision_score( y_train, cbbb.predict(x_train))) #Precision: 0.7164179104477612

Accuracy: 0.9625158269780042
F1 score: 0.9481318502220594
Recall: 0.9173697074824568
Precision: 0.9810286677908938
Wall time: 205 ms


In [66]:
%%time
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score
print("Accuracy:" ,accuracy_score(  y_test, cbbb.predict(x_test))) #Accuracy: 0.998776450266302
print("F1 score:" ,f1_score(        y_test, cbbb.predict(x_test))) #F1 score: 0.37065637065637064
print("Recall:"   ,recall_score(    y_test, cbbb.predict(x_test))) #Recall: 0.25
print("Precision:",precision_score( y_test, cbbb.predict(x_test))) #Precision: 0.7164179104477612

Accuracy: 0.9554770318021202
F1 score: 0.93890415387102
Recall: 0.9085079762277135
Precision: 0.9714046822742475
Wall time: 166 ms


In [67]:
%%time
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score
print("Accuracy:" ,accuracy_score(  y_out, cbbb.predict(x_out))) #Accuracy: 0.998776450266302
print("F1 score:" ,f1_score(        y_out, cbbb.predict(x_out))) #F1 score: 0.37065637065637064
print("Recall:"   ,recall_score(    y_out, cbbb.predict(x_out))) #Recall: 0.25
print("Precision:",precision_score( y_out, cbbb.predict(x_out))) #Precision: 0.7164179104477612

Accuracy: 0.9527707437724515
F1 score: 0.9349342852506897
Recall: 0.9011573350015639
Precision: 0.971341874578557
Wall time: 96.1 ms


## Leitura dos dados originais

In [68]:
%%time
df_all = pd.read_pickle("perfilpj.pkl")

Wall time: 177 ms


In [97]:
df_all[atributes].columns

Index(['NR_CPR', 'CNAE', 'CD_NVL_RCO', 'NATJUR', 'PORTE', 'QTDSOCIOS',
       'ln_idade', 'ln_tempo', 'ln_renda', 'ln_serasa', 'ln_disponivel',
       'ln_transacao', 'ln_capsoc_sic', 'ln_capsoc_rec', 'ln_ativoimov',
       'ln_socios', 'diff_idade_tempo', 'diff_renda_disp', 'diff_renda_tran',
       'diff_renda_cap', 'diff_renda_ativo'],
      dtype='object')

In [98]:
len(df_all[atributes].columns)

21

In [71]:
df_all['NR_CPR']      = df_all['NR_CPR'].astype(int)
df_all['PORTE']       = df_all['PORTE'].astype(int)
df_all['NATJUR']      = df_all['NATJUR'].astype(int) 
df_all['CNAE']        = df_all.CNAE.astype("category").cat.codes
df_all['CD_NVL_RCO']  = df_all.CD_NVL_RCO.astype("category").cat.codes
df_all['QTDSOCIOS']   = df_all['QTDSOCIOS'].astype(int) 

In [72]:
## Dado original
#X0_mm = scaler.transform(df[atributes])
X0_mm = df_all[atributes]

In [73]:
#y0 = df['fraude']
print(X0_mm.shape, df.shape)

(44691, 21) (44691, 202)


In [74]:
%%time
#df_all['proba'] = cbbb.predict_proba(X0_mm)[:,1]
df_all['proba'] = cbbb.predict_proba(df_all[atributes])[:,1]

Wall time: 55.1 ms


In [75]:
df_all[['FRAUDES','proba']].groupby(['FRAUDES']).count()

,proba
FRAUDES,
0.0,42453
1.0,2238


In [76]:
%%time
df_all[['FRAUDES','proba']].groupby(['FRAUDES']).describe()

Wall time: 29.4 ms


proba                                                              \
           count      mean       std       min       25%       50%       75%   
FRAUDES                                                                        
0.0      42453.0  0.066089  0.104925  0.000013  0.012068  0.031692  0.074030   
1.0       2238.0  0.594339  0.414987  0.000937  0.101631  0.841280  0.986915   

                   
              max  
FRAUDES            
0.0      0.996629  
1.0      0.999958

In [129]:
%%time
df_all.loc[ df_all['proba']>=0.15 ,'fraude_pred'] = 1
df_all.loc[ df_all['proba'] <0.15 ,'fraude_pred'] = 0

Wall time: 5.98 ms


<a name="metricas"></a>


# 3) Métricas
  
- ir para [Menu Principal](#navegacao)

<a name="amostra"></a>


## 3.1) Métricas na Amostra
  
- ir para [Menu Principal](#navegacao)

In [130]:
from sklearn.metrics import confusion_matrix
confusao_pop = confusion_matrix(df_all['FRAUDES'], df_all['fraude_pred'])
print("Confusion matrix for test:\n%s" % confusao_pop )

Confusion matrix for test:
[[37964  4489]
 [  662  1576]]


In [131]:
%%time
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score

print("Accuracy:" , accuracy_score(df_all['FRAUDES'], df_all['fraude_pred'])) 
print("F1 score:" , f1_score(df_all['FRAUDES'], df_all['fraude_pred'])) 
print("Recall:"   , recall_score(df_all['FRAUDES'], df_all['fraude_pred'])) 
print("Precision:", precision_score(df_all['FRAUDES'], df_all['fraude_pred'])) 

Accuracy: 0.8847418943411425
F1 score: 0.3796218234373118
Recall: 0.7042001787310098
Precision: 0.2598516075845012
Wall time: 73.8 ms


<a name="falsopos"></a>

## 3.2) Taxa de Falso Positivo
  
- ir para [Menu Principal](#navegacao)

In [132]:
%%time
tn, fp, fn, tp = confusao_pop.ravel()
print('FPR:', fp/(fp + tn))
print('TPR:', tp/(tp + fn))

#FPR: 0.10574046592702518
#TPR: 0.7042001787310098

FPR: 0.10574046592702518
TPR: 0.7042001787310098
Wall time: 985 µs
